<h1><center> IT550: Information Retrieval<h1>
<h2> <center> Assignment 5: Document Retrieval and Evaluation (3) </h2>


<h4> <center> <b>Name : Arjun Vankani </h4>
<h4> <b> <center>ID : 202211036 </h4>

<b><h1>Q-1). Load the MS MARCO dataset and index it

**Downloading Data set and Extracting**

In [ ]:
!pip install python-terrier

**Importing library**

In [ ]:
import pyterrier as pt
import pandas as pd

**Intilize pyterrior**

In [ ]:
if not pt.started():
  pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



**Loading dataset**

In [ ]:
ms_dataset = pt.get_dataset("trec-deep-learning-docs")

In [ ]:
ms_dataset

RemoteDataset for msmarco_document, with ['corpus', 'corpus-tsv', 'topics', 'qrels', 'info_url', 'corpus_iter', 'index']

**Taking title and body objects/rows**

In [ ]:
data = ["title", "body"]
indexer_with_fields = pt.TRECCollectionIndexer("./index_with_fields")

**Set the properties on Terrier's global propertie**


In [ ]:
pt.ApplicationSetup.setProperty("indexer.meta.forward.keys", ",".join(data))
pt.ApplicationSetup.setProperty("indexer.meta.reverse.keys", ",".join(data))
pt.ApplicationSetup.setProperty("FieldTags.process", ",".join(data))
pt.ApplicationSetup.setProperty("TrecDocTags.doctag", "DOC")
pt.ApplicationSetup.setProperty("TrecDocTags.idtag", "DOCNO")
pt.ApplicationSetup.setProperty("TrecDocTags.skip", "DOCHDR")

**Indexing**

In [ ]:
indexref_with_fields = indexer_with_fields.index(ms_dataset.get_corpus())
index_with_fields = pt.IndexFactory.of(indexref_with_fields)

In [ ]:
topics = ms_dataset.get_topics("test")
topics

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

,qid,query
0,1108939,what slows down the flow of blood
1,1112389,what is the county for grand rapids mn
2,792752,what is ruclip
3,1119729,what do you do when you have a nosebleed from ...
4,1105095,where is sugar lake lodge located
...,...,...
195,146187,difference between a mcdouble and a double che...
196,634428,what does chs stand for
197,1121986,what are the effects of having low blood sugar
198,321441,how much is a us postal stamp cost


In [ ]:
qrels = ms_dataset.get_qrels("test")
qrels

2019qrels-docs.txt:   0%|          | 0.00/331k [00:00<?, ?iB/s]

,qid,docno,label
0,19335,D1035833,0
1,19335,D1068065,0
2,19335,D1068066,0
3,19335,D1088,0
4,19335,D109632,2
...,...,...,...
16253,1133167,D925688,0
16254,1133167,D936636,2
16255,1133167,D938771,0
16256,1133167,D973264,1


**Define the models**

In [ ]:
models = {
    "TF-IDF": pt.BatchRetrieve(index_with_fields, wmodel="TF_IDF"),
    "BM25": pt.BatchRetrieve(index_with_fields, wmodel="BM25"),
    "BM25F": pt.BatchRetrieve(index_with_fields, wmodel="BM25F"),
    "PL2": pt.BatchRetrieve(index_with_fields, wmodel="PL2"),
    "PL2F": pt.BatchRetrieve(index_with_fields, wmodel="PL2F"),
    "DPH": pt.BatchRetrieve(index_with_fields, wmodel="DPH")
}

In [ ]:
models

{'TF-IDF': BR(/content/index_with_fields/data.properties,{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'wmodel': 'TF_IDF'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.controls': 'wmodel:DPH,parsecontrols:on,parseql:on,applypipeline:on,terrierql:on,localmatching:on,filters:on,decorate:on', 'querying.allowed.controls': 'scope,qe,qemodel,start,end,site,scope,applypipeline', 'termpipelines': 'Stopwords,PorterStemmer'}),
 'BM25': BR

**Evaluate the results using the specified metrics**

In [ ]:
evalulation_metrics = ["map_cut_10", "map_cut_100", "ndcg_cut_10", "ndcg_cut_100"]

In [ ]:
results = {}
for name, model in models.items():
    results[name] = model.transform(topics)

evaluator = {}
for model_name, model_results in results.items():
    evaluator[model_name] = pt.Utils.evaluate(model_results, qrels, evalulation_metrics)

**Calculating MRR@10 and MRR@100**

In [ ]:
def calculate_reciprocal_rank(results_dict, qrels):
    metrics = ["recip_rank"]
    mrr_eval = {}

    for model_name, results in results_dict.items():
        top_10_results = results.groupby('query').head(10)
        evaluator_10 = pt.Utils.evaluate(top_10_results, qrels, metrics=metrics)

        top_20_results = results.groupby('query').head(20)
        evaluator_20 = pt.Utils.evaluate(top_20_results, qrels, metrics=metrics)

        #top_50_results = results.groupby('query').head(50)
        #evaluator_50 = pt.Utils.evaluate(top_50_results, qrels, metrics=metrics)


        #top_100_results = results.groupby('query').head(100)
        #evaluator_100 = pt.Utils.evaluate(top_100_results, qrels, metrics=metrics)

        recip_rank_10 = evaluator_10["recip_rank"]
        recip_rank_20 = evaluator_20["recip_rank"]
        #recip_rank_50 = evaluator_50["recip_rank"]
        #recip_rank_100 = evaluator_100["recip_rank"]

        mrr_eval[model_name] = {
            "MRR@10": recip_rank_10,
            "MRR@20": recip_rank_20,
            #"MRR@50": recip_rank_50,
            #"MRR@100": recip_rank_100
        }

    return mrr_eval


In [ ]:
mrr_eval = calculate_reciprocal_rank(results, qrels)

In [ ]:
df = pd.DataFrame.from_dict(evaluator).T
df.columns = ["MAP@10","MAP@20", "nDCG@10", "nDCG@20"]
df_rank = pd.DataFrame.from_dict(mrr_eval).T


In [ ]:
merged_data = pd.concat([df_rank, df], axis=1)
merged_data

,MRR@10,MRR@20,MAP@10,MAP@20,nDCG@10,nDCG@20
TF-IDF,0.864987,0.864987,0.113309,0.244909,0.524194,0.503096
BM25,0.874123,0.874123,0.114305,0.248334,0.538802,0.512328
BM25F,0.023256,0.023256,0.023256,0.023259,0.018011,0.018193
PL2,0.841085,0.842746,0.112509,0.248535,0.542381,0.518715
PL2F,0.023256,0.023256,0.023256,0.023259,0.018011,0.018197
DPH,0.907946,0.907946,0.115437,0.249372,0.543129,0.515971


**for MRR@50 and @MRR100**

In [ ]:
def calculate_reciprocal_rank(results_dict, qrels):
    metrics = ["recip_rank"]
    mrr_eval = {}

    for model_name, results in results_dict.items():
        #top_10_results = results.groupby('query').head(10)
        #evaluator_10 = pt.Utils.evaluate(top_10_results, qrels, metrics=metrics)

        #top_20_results = results.groupby('query').head(20)
        #evaluator_20 = pt.Utils.evaluate(top_20_results, qrels, metrics=metrics)

        top_50_results = results.groupby('query').head(50)
        evaluator_50 = pt.Utils.evaluate(top_50_results, qrels, metrics=metrics)


        top_100_results = results.groupby('query').head(100)
        evaluator_100 = pt.Utils.evaluate(top_100_results, qrels, metrics=metrics)

        #recip_rank_10 = evaluator_10["recip_rank"]
        #recip_rank_20 = evaluator_20["recip_rank"]
        recip_rank_50 = evaluator_50["recip_rank"]
        recip_rank_100 = evaluator_100["recip_rank"]

        mrr_eval[model_name] = {
            #"MRR@10": recip_rank_10,
            #"MRR@20": recip_rank_20,
            "MRR@50": recip_rank_50,
            "MRR@100": recip_rank_100
        }

    return mrr_eval


In [ ]:
mrr_eval = calculate_reciprocal_rank(results, qrels)

In [ ]:
df = pd.DataFrame.from_dict(evaluator).T
df.columns = ["MAP@50","MAP@50", "nDCG@100", "nDCG@100"]
df_rank = pd.DataFrame.from_dict(mrr_eval).T


In [ ]:
merged_data = pd.concat([df_rank, df], axis=1)
merged_data

,MRR@50,MRR@100,MAP@50,MAP@50,nDCG@100,nDCG@100
TF-IDF,0.866044,0.866044,0.113309,0.244909,0.524194,0.503096
BM25,0.874123,0.874123,0.114305,0.248334,0.538802,0.512328
BM25F,0.023256,0.023256,0.023256,0.023259,0.018011,0.018193
PL2,0.842746,0.842746,0.112509,0.248535,0.542381,0.518715
PL2F,0.024363,0.024211,0.023256,0.023259,0.018011,0.018197
DPH,0.907946,0.907946,0.115437,0.249372,0.543129,0.515971
